In [61]:
for dir_name in os.listdir(MODELS_DIR):
    metadata = pickle.load(open(os.path.join(MODELS_DIR, dir_name, 'metadata.pickle')))
    print dir_name, '=>', metadata['results']['eval'][-1], metadata['results']['train'][-1], metadata['feature_set'], metadata['params']['colsample_bytree'], metadata['params']['bst:max_depth'], metadata['params']['min_child_weight'], metadata['params']['num_round']

20151013_001704 => 0.71483 0.721357 Train_Standard_With_Dates_And_One_Hot 0.1 12 10 2
20151013_011453 => 0.78964 0.912559 Train_Standard_With_Dates_And_One_Hot 0.1 12 10 1800
20151013_020259 => 0.77951 0.905197 Train_Standard_With_NA_Dates_And_One_Hot 0.1 12 10 200
20151013_021019 => 0.78019 0.902086 Train_Standard_With_NA_Dates_And_One_Hot 0.1 12 10 200
20151013_053542 => 0.79421 0.976206 Train_Standard_With_NA_Dates_And_One_Hot 0.25 12 10 3000
20151013_110751 => 0.79498 0.999864 Train_Standard_With_NA_Dates_And_One_Hot 0.25 12 10 3000
20151013_224302 => 0.79415 0.999903 Train_Standard_With_NA_Dates_And_One_Hot 0.1 12 5 1000
20151013_233726 => 0.79314 0.990197 Train_Standard_With_NA_Dates_And_One_Hot 0.1 12 20 1000
20151014_220031 => 0.73387 0.798566 Train_Standard_With_NA_Dates_And_One_Hot 0.1 12 20 2
20151014_221408 => 0.73387 0.798566 Train_Standard_With_NA_Dates_And_One_Hot 0.1 12 20 2
20151014_221540 => 0.73387 0.798566 Train_Standard_With_NA_Dates_And_One_Hot 0.1 12 20 2
2015101

In [8]:
MODEL_FILE_1 = '20151014_233700'
MODEL_FILE_2 = '20151015_005315'

model_1 = xgb.Booster({'nthread':2})
model_2 = xgb.Booster({'nthread':2})

model_mdata_1 = pickle.load(open(os.path.join(MODELS_DIR, MODEL_FILE_1, 'metadata.pickle')))
model_mdata_2 = pickle.load(open(os.path.join(MODELS_DIR, MODEL_FILE_2, 'metadata.pickle')))

model_1.load_model(os.path.join(MODELS_DIR, MODEL_FILE_1, 'xgboost.model'))
model_2.load_model(os.path.join(MODELS_DIR, MODEL_FILE_2, 'xgboost.model'))

feats1 = model_1.get_fscore()
feats2 = model_2.get_fscore()

In [11]:
def map_features(raw_features_dict, features_names):
    features_data = {}

    for i in range(0, len(features_names)):
        key = 'f{}'.format(i)
        if key in raw_features_dict:
            features_data[features_names[i]] = raw_features_dict[key]
        else:
            features_data[features_names[i]] = 0
    return features_data

In [19]:
features_1 = pd.Series(map_features(feats1, model_mdata_1['columns']))
features_2 = pd.Series(map_features(feats2, model_mdata_2['columns']))

In [24]:
data_for_df = {'model_1':features_1, 'model_2': features_2}
features_data = pd.DataFrame(data_for_df)

In [29]:
features_data.sum()

model_1    508590
model_2    496460
dtype: float64

In [30]:
features_data[features_data.index.isin(['VAR0073','VAR0075','VAR0156','VAR0157','VAR0158','VAR0159','VAR0166','VAR0167','VAR0168','VAR0169','VAR0176','VAR0177','VAR0178','VAR0179','VAR0204','VAR0217'])]

,model_1,model_2
VAR0073,999,664
VAR0075,1369,1255
VAR0156,89,29
VAR0157,13,0
VAR0158,17,7
VAR0159,98,48
VAR0166,318,94
VAR0167,15,4
VAR0168,157,69
VAR0169,241,81


In [60]:
dates = ['VAR0073','VAR0075','VAR0156','VAR0157','VAR0158','VAR0159','VAR0166','VAR0167','VAR0168','VAR0169','VAR0176','VAR0177','VAR0178','VAR0179','VAR0204','VAR0217']
total_raw = total = total_dow = total_dom = total_doy = total_mon = 0
for ddate in dates:
    total_raw += features_data[features_data.index == ddate].sum()[1]
    total += features_data[features_data.index.str.startswith(ddate)].sum()[1]
    total_dow += features_data[features_data.index.str.startswith(ddate.replace('VAR', 'DOW'))].sum()[1]
    total_dom += features_data[features_data.index.str.startswith(ddate.replace('VAR', 'DOM'))].sum()[1]
    total_doy += features_data[features_data.index.str.startswith(ddate.replace('VAR', 'DOY'))].sum()[1]
    total_mon += features_data[features_data.index.str.startswith(ddate.replace('VAR', 'MON'))].sum()[1]
print total_raw, total, total_dow, total_dom, total_doy, total_mon

3880.0 20557.0 1393.0 2785.0 3708.0 1441.0


In [54]:
features_data.loc['VAR0204H']

model_1     NaN
model_2    1045
Name: VAR0204H, dtype: float64